In [136]:
import pandas as pd
import numpy as np

In [37]:
import os

In [38]:
import sqlite3

In [63]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from lxml import etree

In [59]:
DATABASE_PATH = os.getcwd() + '/sages_data_sources/tests/sql/titanic.db'

In [60]:
DATABASE_PATH

'/home/mikolaj/DS/Sages/zjazd_4/sages_data_sources/tests/sql/titanic.db'

In [56]:
con = sqlite3.connect(DATABASE_PATH)

In [57]:
con

In [43]:
def run_query(query):
    con = sqlite3.connect(DATABASE_PATH)
    try:
        response = pd.read_sql_query(query, con)
        return response
    finally:
        con.close()

In [44]:
titanic_data = pd.read_csv('sages_data_sources/tests/sql/input_data/sql/titanic.csv')

survivors = titanic_data[['PassengerId', 'Survived']]
tickets = titanic_data[['PassengerId', 'Ticket', 'Pclass', 'Fare', 'Cabin', 'Embarked']]
passengers = titanic_data[['PassengerId', 'Name', 'Sex', 'Age', 'SibSp', 'Parch']]

In [61]:
try:
    survivors.to_sql('survivors', con, if_exists='replace')
    tickets.to_sql('tickets', con, if_exists='replace')
    passengers.to_sql('passengers', con, if_exists='replace')
finally:
    con.close()

In [228]:
Names_sql = run_query("SELECT p.Name FROM passengers p JOIN survivors s ON p.PassengerId = s.PassengerId WHERE p.Age > 35 AND s.Survived = 1")["Name"].tolist()

In [339]:
query = run_query("SELECT * FROM passengers p JOIN survivors s ON p.PassengerId = s.PassengerId WHERE p.Age > 35 AND s.Survived = 1")

In [351]:
query = query.drop("index", axis=1)
query

,PassengerId,Name,Sex,Age,SibSp,Parch,PassengerId,Survived
0,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,2,1
1,12,"Bonnell, Miss. Elizabeth",female,58.0,0,0,12,1
2,16,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,16,1
3,26,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,26,1
4,53,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,53,1
...,...,...,...,...,...,...,...,...
78,858,"Daly, Mr. Peter Denis",male,51.0,0,0,858,1
79,863,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0,0,863,1
80,866,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,866,1
81,872,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,872,1


In [229]:
Names_sql;

In [64]:
URL = 'https://en.wikipedia.org/wiki/Passengers_of_the_Titanic'

In [246]:
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a')

In [83]:
urls = ['https://en.wikipedia.org/wiki' + str(link.get('href')) for link in links]

In [95]:
tables = soup.find_all('table')

In [286]:
wikipedia_url = 'https://en.wikipedia.org/wiki/Passengers_of_the_Titanic'
response = requests.get(wikipedia_url)
response_page = BeautifulSoup(response.text, 'html.parser')
dom_tree = etree.HTML(str(response_page))
xpath = ".//table//tr[contains(@style, 'background')]/td[1]/a"
found_elements = dom_tree.xpath(xpath)

In [258]:
# dfs = [pd.read_html(tables[i].prettify())[0] for i in range(3)]
# df = pd.concat(dfs, ignore_index=False)
# df = df[~df["Lifeboat"].isna()].reset_index(drop=True)
# Names = df["Name"].tolist()

In [296]:
# urls = []
# for name in Names_sql:
#     for link in links:
#         try:
#             if name in str(link):
#                 urls.append(name)
#         except:
#             None
# urls

['Hosono, Mr. Masabumi',
 'Carter, Mr. William Ernest',
 'Harper, Mr. Henry Sleeper']

In [302]:
people_urls = []
for element in found_elements:
    if element.text in Names_sql:
        tpl = (f"https://en.wikipedia.org/{element.attrib['href']}", element.text)
        people_urls.append(tpl)
people_urls

[('https://en.wikipedia.org//wiki/William_E._Carter',
  'Carter, Mr. William Ernest'),
 ('https://en.wikipedia.org//wiki/Henry_S._Harper',
  'Harper, Mr. Henry Sleeper'),
 ('https://en.wikipedia.org//wiki/Masabumi_Hosono', 'Hosono, Mr. Masabumi')]

In [326]:
url = people_urls[0][0]

In [328]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
tree = etree.HTML(str(soup))
table = tree.xpath(".//table[contains(@class, 'infobox')]")
table_html = etree.tostring(table[0])
df = pd.read_html(table_html)[0]
df

,William E. Carter,William E. Carter.1
0,Born,"William Ernest CarterPhiladelphia, Pennsylvani..."
1,Died,"March 20, 1940Palm Beach, Florida, USA"
2,Nationality,American
3,Alma mater,University of Pennsylvania
4,Known for,surviving the RMS Titanic
5,Spouse(s),Lucile Carter


In [318]:
df.to_excel(writer, sheet_name=SHEET_1, index=False)

,William E. Carter,William E. Carter.1
0,Born,"William Ernest CarterPhiladelphia, Pennsylvani..."
1,Died,"March 20, 1940Palm Beach, Florida, USA"
2,Nationality,American
3,Alma mater,University of Pennsylvania
4,Known for,surviving the RMS Titanic
5,Spouse(s),Lucile Carter


In [329]:
people_urls

[('https://en.wikipedia.org//wiki/William_E._Carter',
  'Carter, Mr. William Ernest'),
 ('https://en.wikipedia.org//wiki/Henry_S._Harper',
  'Harper, Mr. Henry Sleeper'),
 ('https://en.wikipedia.org//wiki/Masabumi_Hosono', 'Hosono, Mr. Masabumi')]

In [341]:
query

,index,PassengerId,Name,Sex,Age,SibSp,Parch,index,PassengerId,Survived
0,1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,1,2,1
1,11,12,"Bonnell, Miss. Elizabeth",female,58.0,0,0,11,12,1
2,15,16,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,15,16,1
3,25,26,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,25,26,1
4,52,53,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,52,53,1
...,...,...,...,...,...,...,...,...,...,...
78,857,858,"Daly, Mr. Peter Denis",male,51.0,0,0,857,858,1
79,862,863,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0,0,862,863,1
80,865,866,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,865,866,1
81,871,872,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,871,872,1


In [342]:
query[query["Name"] == 'Carter, Mr. William Ernest']

,index,PassengerId,Name,Sex,Age,SibSp,Parch,index,PassengerId,Survived
28,390,391,"Carter, Mr. William Ernest",male,36.0,1,2,390,391,1


In [352]:
EXCEL_TO_BE_SAVED = 'excel_to_be_saved.xlsx'
writer = pd.ExcelWriter(EXCEL_TO_BE_SAVED, engine='xlsxwriter')

for url, name in people_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tree = etree.HTML(str(soup))
    table = tree.xpath(".//table[contains(@class, 'infobox')]")
    table_html = etree.tostring(table[0])
    df = pd.read_html(table_html)[0]
    df2 = query[query["Name"] == name].transpose().reset_index()
    df.columns = ["a","b"]
    df2.columns = ["a","b"]
    df = df.append(df2)
    df.to_excel(writer, sheet_name=name, index=False, header=False)
    
writer.save()

/tmp/ipykernel_44913/1237064217.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
/tmp/ipykernel_44913/1237064217.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
/tmp/ipykernel_44913/1237064217.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)


In [336]:
EXCEL_TO_BE_SAVED = 'excel_to_be_saved.xlsx'
writer = pd.ExcelWriter(EXCEL_TO_BE_SAVED, engine='xlsxwriter')

for url, name in people_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tree = etree.HTML(str(soup))
    table = tree.xpath(".//table[contains(@class, 'infobox')]")
    table_html = etree.tostring(table[0])
    df = pd.read_html(table_html)[0]
    df.to_excel(writer, sheet_name=name, index=False, header=False)
    
writer.save()